# Generate Symbolic Predictions for Fusion

Run this on Thunder Compute after training the PercePiano symbolic model.
Generates predictions aligned with audio validation samples for fusion testing.

**Output:** `gdrive:crescendai_data/predictions/symbolic_predictions.json`

In [ ]:
# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install rclone
!curl -fsSL https://rclone.org/install.sh | sudo bash 2>&1 | grep -E "(successfully|already)" || echo "rclone installed"

In [ ]:
# Setup PercePiano with numpy 2.0 compatibility
import os
import sys
from pathlib import Path
from types import ModuleType
import subprocess

PERCEPIANO_ROOT = Path('/tmp/PercePiano')
if not PERCEPIANO_ROOT.exists():
    print("Cloning PercePiano repository...")
    !git clone https://github.com/JonghoKimSNU/PercePiano.git /tmp/PercePiano
else:
    print(f"PercePiano already present at {PERCEPIANO_ROOT}")

PERCEPIANO_PATH = PERCEPIANO_ROOT / 'virtuoso' / 'virtuoso'

# Install dependencies
!pip install omegaconf tqdm --quiet

# Patch numpy 2.0 compatibility
import numpy as np
if not hasattr(np.lib, 'arraysetops'):
    arraysetops = ModuleType('numpy.lib.arraysetops')
    arraysetops.isin = np.isin
    sys.modules['numpy.lib.arraysetops'] = arraysetops
    np.lib.arraysetops = arraysetops
    print("Patched numpy.lib.arraysetops for numpy 2.0 compatibility")

sys.path.insert(0, str(PERCEPIANO_PATH / 'pyScoreParser'))
sys.path.insert(0, str(PERCEPIANO_PATH))

print(f"\nnumpy version: {np.__version__}")
print(f"PercePiano path: {PERCEPIANO_PATH}")

In [ ]:
# Download data from GDrive
import subprocess
from pathlib import Path

DATA_ROOT = Path('/tmp/percepiano_original')
CHECKPOINT_ROOT = Path('/tmp/checkpoints/percepiano_original')
FOLD_ASSIGNMENTS_FILE = Path('/tmp/audio_fold_assignments.json')

DATA_ROOT.mkdir(parents=True, exist_ok=True)
CHECKPOINT_ROOT.mkdir(parents=True, exist_ok=True)

# Check rclone
result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)
if 'gdrive:' not in result.stdout:
    raise RuntimeError("rclone not configured. Run 'rclone config' first.")
print("rclone configured")

print("\nDownloading data...")
!rclone copy gdrive:crescendai_data/percepiano_original $DATA_ROOT --progress
!rclone copy gdrive:crescendai_data/checkpoints/percepiano_original $CHECKPOINT_ROOT --progress
!rclone copyto gdrive:crescendai_data/audio_baseline/audio_fold_assignments.json $FOLD_ASSIGNMENTS_FILE

print("\nVerifying...")
print(f"Data folds: {len(list(DATA_ROOT.glob('fold*')))}")
print(f"Checkpoints: {len(list(CHECKPOINT_ROOT.glob('*.pt')))}")
print(f"Fold assignments: {FOLD_ASSIGNMENTS_FILE.exists()}")

In [ ]:
# Load fold assignments
import json

with open(FOLD_ASSIGNMENTS_FILE) as f:
    fold_assignments = json.load(f)

# Get all keys we need predictions for
all_keys = set()
for fold_id in range(4):
    all_keys.update(fold_assignments.get(f'fold_{fold_id}', []))
all_keys.update(fold_assignments.get('test', []))

print(f"Total keys to predict: {len(all_keys)}")
print(f"  Validation: {sum(len(fold_assignments.get(f'fold_{i}', [])) for i in range(4))}")
print(f"  Test: {len(fold_assignments.get('test', []))}")

In [ ]:
# Import PercePiano model
from model_m2pf import VirtuosoNetMultiLevel
from omegaconf import OmegaConf
import yaml
import pickle

# Load config
CONFIG_PATH = PERCEPIANO_PATH.parent / 'ymls' / 'shared' / 'label19' / 'han_measnote_nomask_bigger256.yml'
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

net_param = OmegaConf.create(config['nn_params'])
net_param.graph_keys = []

print(f"Config loaded: {CONFIG_PATH.name}")
print(f"Hidden size: {net_param.encoder.size}")

In [ ]:
# Helper functions
import torch
from torch.nn.utils.rnn import pack_sequence

def extract_label_key(filename):
    """Extract label key from pkl filename."""
    name = filename.replace('.pkl', '').replace('.mid', '')
    if name.startswith('all_2rounds_'):
        name = name[len('all_2rounds_'):]
    return name

def load_sample(pkl_path, max_notes=5000):
    """Load a single sample from pkl file."""
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    
    x = torch.tensor(data['input'], dtype=torch.float32)
    if len(x) > max_notes:
        x = x[:max_notes]
    
    note_locations = {
        'beat': torch.tensor(data['note_location']['beat'][:len(x)], dtype=torch.long),
        'measure': torch.tensor(data['note_location']['measure'][:len(x)], dtype=torch.long),
        'voice': torch.tensor(data['note_location']['voice'][:len(x)], dtype=torch.long),
        'section': torch.tensor(data['note_location']['section'][:len(x)], dtype=torch.long),
    }
    return x, note_locations

def predict_single(model, x, note_locations, device, sigmoid):
    """Generate prediction for a single sample."""
    batch_x = pack_sequence([x], enforce_sorted=True).to(device)
    note_locs = {k: v.unsqueeze(0).to(device) for k, v in note_locations.items()}
    
    with torch.no_grad():
        outputs = model(batch_x, None, None, note_locs)
        pred = sigmoid(outputs[-1]).squeeze(0).cpu().numpy()
    return pred

print("Helper functions defined")

In [ ]:
# Build key -> pkl file mapping
key_to_pkl = {}

for fold_id in range(4):
    fold_path = DATA_ROOT / f'fold{fold_id}'
    for split in ['train', 'valid', 'test']:
        split_path = fold_path / split
        if split_path.exists():
            for pkl_file in split_path.glob('*.pkl'):
                if pkl_file.name != 'stat.pkl':
                    key = extract_label_key(pkl_file.name)
                    if key not in key_to_pkl:  # Keep first occurrence
                        key_to_pkl[key] = pkl_file

print(f"Mapped {len(key_to_pkl)} unique keys to pkl files")

# Check coverage
missing = all_keys - set(key_to_pkl.keys())
if missing:
    print(f"Warning: {len(missing)} keys not found in data")
else:
    print("All keys found in data")

In [ ]:
# Generate predictions for each fold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

sigmoid = torch.nn.Sigmoid()
predictions = {}

for fold_id in range(4):
    print(f"\n{'='*50}")
    print(f"FOLD {fold_id}")
    print('='*50)
    
    # Load checkpoint
    checkpoint_path = CHECKPOINT_ROOT / f'fold{fold_id}_best.pt'
    if not checkpoint_path.exists():
        print(f"Checkpoint not found: {checkpoint_path}")
        continue
    
    # Load fold stats
    fold_path = DATA_ROOT / f'fold{fold_id}'
    with open(fold_path / 'train' / 'stat.pkl', 'rb') as f:
        fold_stats = pickle.load(f)
    
    # Update input size
    net_param.input_size = max(v[1] for v in fold_stats['key_to_dim']['input'].values())
    
    # Load model
    model = VirtuosoNetMultiLevel(net_param, fold_stats, multi_level="total_note_cat")
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Loaded model (R2={checkpoint['r2']:.4f}, epoch {checkpoint['epoch']})")
    
    # Get fold validation keys
    fold_keys = set(fold_assignments.get(f'fold_{fold_id}', []))
    print(f"Generating predictions for {len(fold_keys)} validation samples...")
    
    count = 0
    for key in fold_keys:
        if key not in key_to_pkl:
            continue
        
        x, note_locations = load_sample(key_to_pkl[key])
        pred = predict_single(model, x, note_locations, device, sigmoid)
        predictions[key] = pred.tolist()
        count += 1
    
    print(f"Generated {count} predictions for fold {fold_id}")

In [ ]:
# Generate test set predictions using fold 0 model
test_keys = set(fold_assignments.get('test', []))
print(f"\nGenerating predictions for {len(test_keys)} test samples...")

# Reload fold 0 model
fold_path = DATA_ROOT / 'fold0'
with open(fold_path / 'train' / 'stat.pkl', 'rb') as f:
    fold_stats = pickle.load(f)

net_param.input_size = max(v[1] for v in fold_stats['key_to_dim']['input'].values())
model = VirtuosoNetMultiLevel(net_param, fold_stats, multi_level="total_note_cat")
checkpoint = torch.load(CHECKPOINT_ROOT / 'fold0_best.pt', map_location=device, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)
model.eval()

count = 0
for key in test_keys:
    if key in predictions:  # Skip if already predicted
        continue
    if key not in key_to_pkl:
        continue
    
    x, note_locations = load_sample(key_to_pkl[key])
    pred = predict_single(model, x, note_locations, device, sigmoid)
    predictions[key] = pred.tolist()
    count += 1

print(f"Generated {count} test predictions")
print(f"\nTotal predictions: {len(predictions)}")

In [ ]:
# Save predictions locally
output_path = Path('/tmp/symbolic_predictions.json')
with open(output_path, 'w') as f:
    json.dump(predictions, f)

print(f"Saved {len(predictions)} predictions to {output_path}")

In [ ]:
# Upload to GDrive
!rclone mkdir gdrive:crescendai_data/predictions
!rclone copyto /tmp/symbolic_predictions.json gdrive:crescendai_data/predictions/symbolic_predictions.json

print("\nUploaded to gdrive:crescendai_data/predictions/symbolic_predictions.json")

In [ ]:
# Summary
print("="*60)
print("PREDICTION GENERATION COMPLETE")
print("="*60)

val_count = sum(1 for k in predictions if k not in test_keys)
test_count = sum(1 for k in predictions if k in test_keys)

print(f"\nTotal predictions: {len(predictions)}")
print(f"  Validation: {val_count}")
print(f"  Test: {test_count}")
print(f"\nOutput: gdrive:crescendai_data/predictions/symbolic_predictions.json")
print("\nNext: Run 'uv run python scripts/analyze_models.py' locally")